# Assignment 2 (Part 1)

<B>In this assignment, it will be required of you to cluster and explore the neighbourhoods in Toronto</B>

In [1]:
# Import Libraries

import pandas as pd
import requests
import lxml.html as lh

In [2]:
# Scraping and loading the following Wikipedia page

URL_Link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Page = requests.get(URL_Link) # to handle the contents of the website

Documents = lh.fromstring(Page.content) # to store the website contents in a document

TR_Elements = Documents.xpath('//tr') # to parse data that is stored in the <tr>...</tr> elememts

[len(T) for T in TR_Elements[:12]] # to display the first 12 rows

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
# Create empty list and i is the index of our column

col = []
i = 0

# Iterate and store each first element (header) in each row in the empty list

for x in TR_Elements[0]:
    i+=1
    name = x.text_content()
    print ('%d: %s'%(i,name))
    col.append((name,[]))

1: Postal code

2: Borough

3: Neighborhood



In [6]:
# Create a Dataframe using pandas

# Since the first row is the header, data is stored on the second row and onwards

for y in range(1,len(TR_Elements)):
    T = TR_Elements[y] # T is our y'th row
    
    if len(T)!=3: # If row is not of size 3, the //tr data is not from our table 
        break
    
    i = 0 
    
    for z in T.iterchildren():
        data = z.text_content()
        
        if i>0:
            try:
                data=int(data)
            except:
                pass
        
        col[i][1].append(data)
        i+=1
        
[len(C) for (title,C) in col] # To display the length of each column

[181, 181, 181]

In [7]:
# Creating the Dataframe

Dict = {title:column for (title,column) in col}
Neighbourhood_df = pd.DataFrame(Dict)

Neighbourhood_df.head()

,Postal code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n


In [8]:
# Re-arranging, renaming and cleaning the columns

Neighbourhood_df.columns = ['Borough', 'Neighbourhood','Postcode']

Col_List = Neighbourhood_df.columns.tolist()

Col_List_Add = Col_List[-1:] + Col_List[:-1]

Neighbourhoods_df = Neighbourhood_df[Col_List_Add]

Neighbourhoods_df.head()

,Postcode,Borough,Neighbourhood
0,\n,M1A\n,Not assigned\n
1,\n,M2A\n,Not assigned\n
2,Parkwoods\n,M3A\n,North York\n
3,Victoria Village\n,M4A\n,North York\n
4,Regent Park / Harbourfront\n,M5A\n,Downtown Toronto\n


In [10]:
# Cleaning the messy string in the Borough column

Neighbourhoods_df = Neighbourhoods_df.replace('\n',' ', regex=True)
Neighbourhoods_df.head()

,Postcode,Borough,Neighbourhood
0,,M1A,Not assigned
1,,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Regent Park / Harbourfront,M5A,Downtown Toronto


In [11]:
# Cells with a Borough that is Not Assigned

Neighbourhoods_df.drop(Neighbourhoods_df.index[Neighbourhoods_df['Borough'] == 'Not assigned'], inplace = True)

# Reset and drop the previous index

Neighbourhoods_df = Neighbourhoods_df.reset_index(drop=True)

# Display the first 10 rows

Neighbourhoods_df.head(10)

,Postcode,Borough,Neighbourhood
0,,M1A,Not assigned
1,,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Regent Park / Harbourfront,M5A,Downtown Toronto
5,Lawrence Manor / Lawrence Heights,M6A,North York
6,Queen's Park / Ontario Provincial Government,M7A,Downtown Toronto
7,,M8A,Not assigned
8,Islington Avenue,M9A,Etobicoke
9,Malvern / Rouge,M1B,Scarborough


In [12]:
# Combining Neighbourhoods based on similar Postcode and Borough

Neighbourhoods_df = Neighbourhoods_df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
Neighbourhoods_df.columns = ['Postcode','Borough','Neighbourhood']

# Removing any space in the start of the string

Neighbourhoods_df['Neighbourhood'] = Neighbourhoods_df['Neighbourhood'].str.strip()

# Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"

Neighbourhoods_df.loc[Neighbourhoods_df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = Neighbourhoods_df['Borough']

# Check the shape of the data frame

Neighbourhoods_df.shape

(181, 3)